<a href="https://colab.research.google.com/github/CWaiwit/MADT7202-SNA_Mockup-red01/blob/main/SNA_mockup_03_red01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd;
import numpy as np;

#Mockup Class

##Mock up affiliation network

In [ ]:
class MockupAffNW:
    RAND_SEED = 94232;
    N_USERS = 75;
    N_EXPRESSION = 7501;

    users:pd.DataFrame = None;
    interests:pd.DataFrame = None;
    user_interest:pd.DataFrame = None;
    linkages:pd.DataFrame = None;

    def set_param(self, rand_seed:int=94232, n_user:int=75, n_expr=7501):
        self.RAND_SEED=rand_seed;
        self.N_USERS=n_user;
        self.N_EXPRESSION=n_expr;
        pass;

    def get_interests(self) -> pd.DataFrame:
        if self.interests is None:
            interest = [
                {'interest':'Love', 'topic':'Love'},
                {'interest':'Wealth', 'topic':'Wealth'},
                {'interest':'Career', 'topic':'Career'},
                {'interest':'Health', 'topic':'Others'},
                {'interest':'Birth', 'topic':'Others'},
                {'interest':'Eduction', 'topic':'Others'},
                {'interest':'Romance', 'topic':'non-mu-topic'},
                {'interest':'Pet', 'topic':'non-mu-topic'},
                {'interest':'Lottery', 'topic':'non-mu-topic'},
                {'interest':'Music', 'topic':'non-mu-topic'},
                {'interest':'Weather', 'topic':'non-mu-topic'},
                {'interest':'News', 'topic':'non-mu-topic'},
                {'interest':'Television', 'topic':'non-mu-topic'},
                {'interest':'Leisure', 'topic':'non-mu-topic'}
            ];
            self.interests = pd.DataFrame(interest);
        return self.interests.loc[:];
    
    def get_users(self) -> pd.DataFrame:
        if self.users is None:
            interests:pd.DataFrame = self.get_interests();

            np.random.seed(self.RAND_SEED);
            df:pd.DataFrame = pd.DataFrame(['UID'+str(x+1).zfill(4) for x in range(self.N_USERS)], columns=['uid']);

            # Probability of being detected in users network
            df['p_obsrv'] = np.random.normal(0.5, 0.125, len(df.index));
            obsrv_total = df['p_obsrv'].sum()
            df['p_obsrv'] = df['p_obsrv']/obsrv_total;

            # Generate user's interest
            df['n_interest'] = np.random.randint(3, 7, len(df.index));
            df['n_interest'] = df['n_interest'].round();
            df['interests'] = df['n_interest'].apply(lambda x: np.random.choice(interests['interest'], int(x), replace=False ) );
            df['interests'] = df['interests'].apply(','.join);
            df.drop(columns=['n_interest'], inplace=True);

            self.users = df;
        return self.users.loc[:];

    def get_user_interest(self) -> pd.DataFrame:
        if self.user_interest is None:
            users = self.get_users();
            interest = self.get_interests();
            
            users['interests'] = users['interests'].str.split(',');
            users['x'] = users.apply(lambda x: [{'uid':x.uid, 'interest':i} for i in x.interests ], axis='columns');
            df:pd.DataFrame = pd.DataFrame(users['x'].explode().to_list());

            # self.user_interest = pd.DataFrame(users['x'].explode().to_list());
            df = df.set_index('interest').join(interest.set_index('interest'));
            df.reset_index('interest', inplace=True);

            df.sort_values(['uid', 'interest', 'topic'], ignore_index=True, inplace=True);

            self.user_interest = df[['uid', 'interest', 'topic']];
        return self.user_interest.loc[:];

    def get_linkage(self) -> pd.DataFrame:
        if self.linkages is None:
            users = self.get_users();
            users['interests'] = users['interests'].str.split(',');

            uxi:pd.DataFrame = self.get_user_interest();
            ixu:pd.DataFrame = uxi.groupby('interest')['uid'].apply(set).reset_index();

            # generate src of random linkage
            np.random.seed(self.RAND_SEED);
            df:pd.DataFrame = pd.DataFrame(np.random.choice(users['uid'], self.N_EXPRESSION, p=users['p_obsrv']), columns=['uid_src']);
            
            # random linkage-affiliation
            users.set_index('uid', inplace=True);
            df['interest'] = df['uid_src'].apply(lambda x: np.random.choice(users.at[x, 'interests']));
            users.reset_index(names=['uid'], inplace=True);

            # random affiliated-target
            ixu.set_index('interest', inplace=True);
            df['uid_tgt_ls'] = df.apply(lambda x: sorted(ixu.at[x.interest, 'uid'] - {x.uid_src}), axis='columns');
            df['uid_tgt_ls'] = df['uid_tgt_ls'].apply(list);
            ixu.reset_index(names=['interest'], inplace=True);
            df['uid_tgt_p_ls'] = df.apply(lambda x: users[users['uid'].isin(x['uid_tgt_ls'])]['p_obsrv'].to_list(), axis='columns');
            df['uid_tgt_p_total'] = df['uid_tgt_p_ls'].apply(sum);
            df['uid_tgt_p_ls'] = df.apply(lambda x: [i/x.uid_tgt_p_total for i in x.uid_tgt_p_ls], axis='columns');
            df['uid_tgt'] = df.apply(lambda x: np.random.choice(x.uid_tgt_ls, p=x.uid_tgt_p_ls), axis='columns');
            df.drop_duplicates(subset=['uid_src', 'interest', 'uid_tgt'], inplace=True);
            df.drop(columns=['uid_tgt_ls', 'uid_tgt_p_ls','uid_tgt_p_total'], inplace=True);
            
            df.sort_values(by=['uid_src','uid_tgt', 'interest'], ignore_index=True, inplace=True);

            self.linkages = df[['uid_src','uid_tgt', 'interest']];
        return self.linkages.loc[:];

##Mock up SNA

In [ ]:
class MockupSNA03:
    RAND_SEED = 94232;
    N_USERS = 75;
    N_INFLUENCERS = 25;
    N_BRANDS = 4;
    N_POST = 1501;
    N_INTERACT_LIKE_MIN = 100;
    N_INTERACT_LIKE_MAX = 200;
    N_INTERACT_MIN = 70;
    N_INTERACT_MAX = 156;
    N_RAND_FOLLOW_MIN = 250;
    N_RAND_FOLLOW_MAX = 500;



    users: pd.DataFrame = None;
    user_x_user: pd.DataFrame = None;
    brands: pd.DataFrame = None;
    pages: pd.DataFrame = None;
    brand_x_page: pd.DataFrame =None;


    user_x_page: pd.DataFrame = None;

    post_behavior: pd.DataFrame = None;
    posts: pd.DataFrame = None;
    post_interaction: pd.DataFrame = None;
    
    def set_param(self, seed:int=94232,
            n_like_min:int=100, n_like_max:int=200,
            n_intrx_min:int=70, n_intrx_max:int=156,
            n_follow_min:int=250, n_follow_max:int=500):
        self.RAND_SEED = seed;
        self.N_INTERACT_LIKE_MIN = n_like_min;
        self.N_INTERACT_LIKE_MAX = n_like_max;
        self.N_INTERACT_MIN = n_intrx_min;
        self.N_INTERACT_MAX = n_intrx_max;
        self.N_RAND_FOLLOW_MIN = n_follow_min;
        self.N_RAND_FOLLOW_MAX = n_follow_max;
        pass;

    def gen_model(self):
        '''Generat all data at once'''
        self.get_users();
        self.get_user_x_user();
        self.get_brands()
        self.get_pages();
        self.get_brand_x_page();
        self.get_post_behavior();
        self.get_posts();
        self.get_post_interaction();
        self.get_user_x_page();
        pass;
    
    def init_aff_network(self, n_expr:int=7501, aff:MockupAffNW=None):
        if aff is None:
            aff = MockupAffNW();
            aff.set_param(rand_seed=self.RAND_SEED, n_user=self.N_USERS, n_expr=n_expr);
        
        users = aff.get_users();
        users['interests'] = users['interests'].str.split(',');

        # generate user and preference
        interest = aff.get_interests();
        interest.set_index('interest', inplace=True);
        users['topics'] = users['interests'].apply(lambda x: [ interest.at[i, 'topic'] for i in x]);
        interest.reset_index(names=['interest'], inplace=True);
        users['uxt'] = users[['uid', 'topics']].apply(lambda x: [{'uid':x.uid, 'topic':i} for i in x.topics], axis='columns');
        uxt:pd.DataFrame = pd.DataFrame(users['uxt'].explode().to_list());
        uxt['n'] = 1;
        uxt = uxt.groupby(['uid', 'topic'])[['n']].sum().reset_index();
        uxt_total = uxt.groupby(['uid'])['n'].sum();
        uxt = uxt.set_index('uid').join(uxt_total, rsuffix='_total');
        uxt['topic_p'] = uxt['n']/uxt['n_total'];
        tmp = uxt.groupby('uid')['topic_p'].apply(list).reset_index();
        uxt = uxt.groupby('uid')['topic'].apply(list).reset_index();
        uxt = uxt.set_index('uid').join(tmp.set_index('uid'));
        uxt.reset_index(names='uid', inplace=True);
        uxt['topic'] = uxt['topic'].apply(','.join);
        uxt['topic_p'] = uxt['topic_p'].apply(lambda x: [str(i) for i in x]);
        uxt['topic_p'] = uxt['topic_p'].apply(','.join);
        self.users = uxt;

        # generate user-linkage
        links = aff.get_linkage()
        links = links.set_index('interest').join(interest.set_index('interest'));
        links.reset_index(drop=True, inplace=True);
        links['set'] = links.apply(lambda x: sorted({x.uid_src, x.uid_tgt}), axis='columns');
        links['set'] = links['set'].apply('|'.join);
        links = links.groupby(['set'])['topic'].apply(set).reset_index();
        links['set'] = links['set'].str.split('|');
        links['source'] = links['set'].apply(lambda x: x[0]);
        links['target'] = links['set'].apply(lambda x: x[1]);
        links['linkage'] = links['topic'].apply(','.join);
        links.drop(columns=['set', 'topic'], inplace=True);
        # links.drop(columns=['set', 'topic', 'linkage'], inplace=True);
        # links.drop_duplicates(inplace=True);
        self.user_x_user = links;
    
    def get_users(self):
        '''Generate fb users'''
        if self.users is None:
            df:pd.DataFrame = pd.DataFrame(['UID'+str(x+1).zfill(4) for x in range(self.N_USERS)], columns=['uid']);
            self.users = df;
        return self.users.loc[:];
    
    def get_user_x_user(self) -> pd.DataFrame: return self.user_x_user.loc[:];

    def get_brands(self) -> pd.DataFrame:
        if self.brands is None:
            ls = [
                { 'id': 'Leila', 'type': 'org-brand', 'mrkt_value': 30 },
                { 'id': 'Ravipa', 'type': 'cmpt-brand', 'mrkt_value': 23 },
                { 'id': 'Hermanstones', 'type':'cmpt-brand', 'mrkt_value': 1.5 },
                { 'id': 'Maron', 'type':'cmpt-brand', 'mrkt_value': 1 }
            ];
            self.brands = pd.DataFrame(ls);
        return self.brands.loc[:];
    
    def get_pages(self) -> pd.DataFrame:
        '''Generate list of fb page - include Mu brands'''
        if self.pages is None:
            ls = [
                {'id': 'Leila-page', 'type': 'org-page', 'brand': 'Leila'},
                {'id': 'Ravipa-page', 'type': 'cmpt-page', 'brand': 'Ravipa'},
                {'id': 'Hermanstones-page', 'type': 'cmpt-page', 'brand':'Hermanstones'},
                {'id': 'Maron-page', 'type': 'cmpt-page', 'brand':'Maron'},
            ] + [ {'id': 'PG'+str(i+1).zfill(3), 'type': 'inf-page' } for i in range(25)];

            np.random.seed(self.RAND_SEED);
            np.random.shuffle(ls);
            df:pd.DataFrame = pd.DataFrame(ls);

            self.pages = df[['id', 'type', 'brand']];
        return self.pages.loc[:];

    def get_brand_x_page(self) -> pd.DataFrame:
        '''Organically generate brand-to-page relationship:
        - One brand may support many influencers
        - One influencer may advertise many brand
        - Stregth Scale represents the strength of brand's influence over influencer (str_scale)
        - [Control] No relation between mu-marketing-player: brand and competitor
        - [Control] No relation between influencers'''
        if self.brand_x_page is None:
            df: pd.DataFrame = self.get_pages();
            
            # Random relation brand-influencer
            ## Random brand propability in relation
            np.random.seed(self.RAND_SEED);
            ls:list = self.get_brands().id.to_list() +['others'];
            ls_p = np.random.normal( 0.5, 0.125, len(ls));
            # ls_p = np.random.normal( 0.5, 0.5, len(ls));
            # ls_p = np.absolute(ls_p);
            p_total = ls_p.sum();
            ls_p = ls_p/p_total;
            ls_p = list(ls_p)

            ## Random page probability in relation
            influencer = df[df['type']=='inf-page'][['id']].loc[:]
            # influencer['p'] = np.random.normal( 0.5, 0.125, len(influencer.index));
            influencer['p'] = np.random.normal( 0.5, 0.5, len(influencer.index));
            influencer['p'] = influencer['p'].abs()
            p_total = influencer.p.sum();
            influencer['p'] = influencer['p']/p_total;

            n = 300;
            src = np.random.choice(ls, n, p=ls_p);
            tgt = np.random.choice(influencer.id, n , p=influencer.p);
            links = pd.DataFrame({'source':src, 'target':tgt});
            links.drop(links[links.source=='others'].index, inplace=True);

            # Calculate relatioship strength
            ## Sum number of connections as strength
            links['n'] = 1;
            links = links.groupby(['source', 'target']).n.sum().reset_index(['source', 'target']);
            links['id'] = links.index + 1;

            ## Convert to proportion of strength toward sponsor brand - strength scale.
            links_w = links.groupby('target').n.sum().reset_index().set_index('target');
            links = links.set_index('target').join(links_w, rsuffix='_total').reset_index('target').sort_values('id');
            links['str_scale'] = links['n']/links['n_total'];

            self.brand_x_page = links[['id', 'source', 'target', 'str_scale']];
        return self.brand_x_page.loc[:];

    def get_post_behavior(self) -> pd.DataFrame:
        '''Generate posting behavior of pages:
        - Page's posting is depended-on probability (post_p)
        - Influencer may or may not post mu-content - depended-on proability (mu_p)
        - Strength Scale of brands influence over the ratio of influencer's mu-content (str_scale)'''
        if self.post_behavior is None:
            pgs:pd.DataFrame = self.get_pages();
            bxp:pd.DataFrame = self.get_brand_x_page();

            np.random.seed(self.RAND_SEED);

            # List of sponsor and strength scale
            ## make list of sponsor sources
            bxp__tgt_grp = bxp.groupby('target');
            page_sponsor = bxp__tgt_grp['source'].apply(list).reset_index();
            page_sponsor.set_index('target',inplace=True);

            ## make list of sponsor strength scale
            page_support_sc = bxp__tgt_grp.str_scale.apply(list).reset_index();
            page_support_sc.set_index('target',inplace=True);
            page_sponsor = page_sponsor.join(page_support_sc);

            ## convert list to string - for data prep
            page_sponsor['source'] = page_sponsor['source'].apply(','.join);
            page_sponsor['str_scale'] = page_sponsor['str_scale'].apply(lambda x: [ str(i) for i in x ]);
            page_sponsor['str_scale'] = page_sponsor['str_scale'].apply(','.join);

            # Random mu-post probability
            page_sponsor['mu_p'] = np.random.normal(0.5, 0.076, len(page_sponsor.index));
            page_sponsor['mu_p'] = page_sponsor['mu_p'].round(2);

            # Assign page's post behavior
            ## Join page with sponsors and sponsors' scale
            pgs = pgs.set_index('id').join(page_sponsor);
            pgs. reset_index(inplace=True);

            ## Assign brand to mu-player's page and non-mu-brand page
            pgs['source'] = pgs[['brand', 'type', 'source']].apply(lambda x: x.brand if x['type'] != 'inf-page' else x.source, axis='columns');
            pgs['source'] = pgs['source'].apply(lambda x: 'other' if pd.isna(x) else x);

            ## Assign str_scale to mu-player's page and non-mu-brand page
            pgs['str_scale'] = pgs[['type', 'str_scale']].apply(lambda x: '1.0' if x['type'] != 'inf-page' else x.str_scale, axis='columns');
            pgs['str_scale'] = pgs['str_scale'].fillna('0.0');

            ## Assign mu-post probability to mu-player's page and non-mu-brand page
            pgs['mu_p'] = pgs[['type', 'mu_p']].apply(lambda x: 1.0 if x['type'] !='inf-page' else x.mu_p, axis='columns');
            pgs['mu_p'] = pgs['mu_p'].fillna(0.0);

            ## Assign post distribution probability
            post_p_ls = np.random.normal(0.5, 0.25, len(pgs.index));
            post_p_ls = post_p_ls - post_p_ls.min();
            post_p_ls = post_p_ls/post_p_ls.sum();
            pgs['post_p'] = post_p_ls;

            self.post_behavior = pgs[['id', 'type', 'source', 'str_scale', 'mu_p', 'post_p']];
        return self.post_behavior.loc[:];

    def get_posts(self):
        if self.posts is None:
            '''Generate posts:
            - Page's posts generate ramdomly - depended on page's posting probability (post_p)
            - Page's posts possibly be mu-content -depended on page's mu probability (mu_p)
            - [Control] MU-content is labeled to "Love", "Wealth", "Career", or "Others"
            - [Control] Non-mu-content will not represent any mu-topic nor mu-brands - defined as "non-mu-topic" and "non-mu-brand" respectively
            '''
            beh = self.get_post_behavior();
            np.random.seed(self.RAND_SEED);

            # Random post ower - page
            posts = pd.DataFrame(np.random.choice(beh.id, self.N_POST, p=beh.post_p), columns=['page_id']);
            posts['id'] = posts.index+1;
            posts['id'] = posts['id'].apply(lambda x: 'POST'+str(x).zfill(6))

            # Random flag mu-product post
            beh.set_index('id', inplace=True);
            posts = posts.set_index('page_id').join(beh[['mu_p']]);
            posts.reset_index(names=['page_id'], inplace=True);
            posts['mu_f'] = posts['mu_p'].apply(lambda x: True if x==1 else False if x==0 else np.random.choice([True, False], 1, [x, 1-x])[0] );

            # Random brand of mu-content
            posts_mu = posts[posts['mu_f']].loc[:];
            posts_mu.set_index('page_id', inplace=True);
            posts_mu = posts_mu.join(beh[['source', 'str_scale']]);
            posts_mu.reset_index(names=['page_id'], inplace=True);
            posts_mu['source'] = posts_mu['source'].str.split(',');
            posts_mu['str_scale'] = posts_mu['str_scale'].str.split(',');
            posts_mu['brand'] = posts_mu[['source', 'str_scale']].apply(
                    lambda x: np.random.choice(x.source, 1, x.str_scale)[0],
                    axis='columns');

            # Random mu-topic
            posts_mu['topic'] = np.random.choice(['Love', 'Wealth', 'Career', 'Others'], len(posts_mu.index));

            # Join post - mu-topic
            posts_mu.set_index('id', inplace=True);
            posts.set_index('id', inplace=True);
            posts = posts.join(posts_mu[['brand', 'topic']]);
            posts.reset_index(names=['id'], inplace=True);

            # Fill Nan with non-mu-topic, non-mu-brand
            posts['topic'].fillna('non-mu-topic', inplace=True);
            posts['brand'].fillna('non-mu-brand', inplace=True);

            posts.sort_values('id', ignore_index=True, inplace=True);
            self.posts = posts[['id', 'page_id', 'topic', 'brand']];
        return self.posts.loc[:];

    def get_user_x_page(self) -> pd.DataFrame:
        '''Generate user's following page relationship
        - Assume user proably follow a page if it has any post relevant to user's interest (Homophily)'''
        if self.user_x_page is None:
            users = self.get_users();
            posts = self.get_posts();

            # Topic-to-page probability
            posts['n'] = 1;
            txp = posts.groupby(['topic', 'page_id'])['n'].sum().reset_index();
            txp_total = txp.groupby('topic')['n'].sum().reset_index();
            txp_total.set_index('topic', inplace=True);
            txp = txp.set_index('topic').join(txp_total, rsuffix='_total');
            txp['p'] = txp['n']/txp['n_total'];
            tmp = txp.groupby('topic')['p'].apply(list).reset_index();
            txp = txp.groupby('topic')['page_id'].apply(list).reset_index();
            txp = txp.set_index('topic').join(tmp.set_index('topic'));
            txp.reset_index(names=['topic'], inplace=True);

            # User-interest probability
            users['topic'] = users['topic'].str.split(',');
            users['topic_p'] = users['topic_p'].str.split(',');
            users['topic_p'] = users['topic_p'].apply(lambda x: [float(i) for i in x]);
            
            # Generate sampling
            np.random.seed(self.RAND_SEED);
            n = np.random.randint(self.N_RAND_FOLLOW_MIN, self.N_RAND_FOLLOW_MAX);
            follow:pd.DataFrame = pd.DataFrame( np.random.choice(users['uid'], n), columns=['uid']);

            users.set_index('uid', inplace=True);
            follow['interest'] = follow['uid'].apply(lambda x: np.random.choice( users.at[x,'topic'], p=users.at[x,'topic_p'] ) );
            users.reset_index(names=['uid'], inplace=True);

            txp.set_index('topic', inplace=True);
            follow['page_id'] = follow['interest'].apply(lambda x: np.random.choice( txp.at[x,'page_id'], p=txp.at[x,'p'] ) );
            txp.reset_index(names=['topic'], inplace=True);

            follow.drop(columns=['interest'], inplace=True);
            follow.drop_duplicates(inplace=True, ignore_index=True);
            follow.sort_values(['uid', 'page_id'], ignore_index=True, inplace=True);
            follow['id'] = follow.index + 1;
            
            self.user_x_page = follow[['id','uid','page_id']];
        return self.user_x_page.loc[:];


    def get_post_interaction(self) -> pd.DataFrame:
        '''Random interaction
        - page's post is open publicly - users able to interact regardless follow page or not, chance of following page is higher
        - [Control] user interact post - like: once, commment: many-time, share: many-time'''

        def fn(users:pd.DataFrame, posts:pd.DataFrame, n:int=5000):
            '''Random interact-decisioning: user possibly interact if content from following page or interesting topic'''

            df:pd.DataFrame = pd.DataFrame(np.random.choice(users['uid'], n, p=users['interact_p']), columns=['uid']);
            df = df.set_index('uid').join(users.set_index('uid'));
            df.reset_index(names=['uid'], inplace=True);
            df['is_relevant'] = np.random.random(n);
            df['is_relevant'] = df['is_relevant']>df['irrelevent_p'];
            df['page_id'] = df['page_id'].astype(str);
            df['page_id'] = df['page_id'].str.split(',');
            df['page_id'] = df[['is_relevant', 'page_id']].apply(lambda x: np.random.choice(x.page_id) if x.is_relevant else None, axis='columns');
            df['topic'] = df['topic'].astype(str);
            df['topic'] = df['topic'].str.split(',');
            df['topic'] = df[['is_relevant', 'topic']].apply(lambda x: np.random.choice(x.topic) if not x.is_relevant else None, axis='columns');

            df['post_id'] = df[['page_id', 'topic']].apply(
                    lambda x: posts[posts.page_id==x.page_id]['id'].to_list() if x.page_id is not None else posts[posts.topic==x.topic]['id'].to_list(),
                    axis='columns');
            df['post_id'] = df['post_id'].apply(lambda x: posts.id.to_list() if len(x)<1 else x);
            df['post_id'] = df['post_id'].apply(np.random.choice);
            return df.loc[:];

    
        if self.post_interaction is None:
            users = self.get_users();
            posts = self.get_posts();
            uxpg = self.get_user_x_page();

            np.random.seed(self.RAND_SEED);

            # Random user's post-interaction behavior
            uxpg['page_id'] = uxpg['page_id'].astype(str);
            uxpg = uxpg.groupby('uid')['page_id'].apply(list).reset_index();
            uxpg['page_id'] = uxpg['page_id'].apply(','.join)
            users = users.set_index('uid').join(uxpg.set_index('uid'));
            users.reset_index(names=['uid'], inplace=True);
            users['irrelevent_p'] = np.random.normal(0.15, 0.0225, len(users.index));
            users['interact_p'] = np.random.normal(0.5, 0.125, len(users.index));
            t = users['interact_p'].sum();
            users['interact_p'] = users['interact_p']/t;

            # Like interaction
            n = np.random.randint(self.N_INTERACT_LIKE_MIN, self.N_INTERACT_LIKE_MAX);
            interact_like = fn(users=users, posts=posts, n=n);
            interact_like = interact_like[['uid','post_id']].drop_duplicates();
            interact_like['like'] = 1;

            # Like interaction
            n = np.random.randint(self.N_INTERACT_MIN, self.N_INTERACT_MAX);
            interact_comment = fn(users=users, posts=posts, n=n);
            interact_comment['comment'] = 1;
            interact_comment = interact_comment.groupby(['uid','post_id'])['comment'].sum().reset_index();

            # Share interaction
            n = np.random.randint(self.N_INTERACT_MIN, self.N_INTERACT_MAX);
            interact_share = fn(users=users, posts=posts, n=n);
            interact_share['share'] = 1;
            interact_share = interact_share.groupby(['uid','post_id'])['share'].sum().reset_index();

            # Summary interaction
            interact_comment = interact_comment.groupby(['uid', 'post_id']).comment.sum().reset_index()
            interact_share = interact_share.groupby(['uid', 'post_id']).share.sum().reset_index()
            interact = pd.concat([interact_like, interact_comment, interact_share]);
            interact.fillna(0, inplace=True);

            interact['id'] = interact.index+1;
            interact['id'] = interact['id'].apply(lambda x: 'RESP'+str(x).zfill(10));

            self.post_interaction = interact[['id', 'uid', 'post_id', 'like', 'comment', 'share']];
        return self.post_interaction.loc[:];

In [ ]:
class GenGraph:

    sna:MockupSNA03 = None;
    def __init__(self):
        sna = MockupSNA03();
        # sna.set_param( n_like_min=500, n_like_max=1000, n_intrx_min=350, n_intrx_max=780, n_follow_min=750, n_follow_max=1250 );
        sna.set_param( n_like_min=350, n_like_max=500, n_intrx_min=225, n_intrx_max=427, n_follow_min=125, n_follow_max=422 );
        # sna.init_aff_network(1751);
        sna.init_aff_network();
        sna.gen_model();
        self.sna = sna;
        pass;

    def get_sna(self) -> MockupSNA03: return self.sna;

    def gen_practice01(self):
        sna = self.sna;

        # Gen edge
        posts = sna.get_posts();
        interact = sna.get_post_interaction();
        interact = interact.set_index('post_id').join(posts.set_index('id'));
        interact.reset_index(names=['post_id'], inplace=True);
        interact['topic'] = interact['topic']+',ads-engagement';

        user_engagement:pd.DataFrame = interact.groupby(['uid', 'page_id'])[['like', 'comment', 'share']].sum().reset_index();
        tmp = interact.groupby(['uid', 'page_id'])['topic'].apply(list).reset_index();
        user_engagement = user_engagement.set_index(['uid', 'page_id']).join(tmp.set_index(['uid', 'page_id']));
        user_engagement.reset_index(names=['uid', 'page_id'], inplace=True);
        user_engagement['topic'] = user_engagement['topic'].apply(','.join);
        user_engagement['topic'] = user_engagement['topic'].astype(str);
        user_engagement['topic'] = user_engagement['topic'].str.split(',');
        user_engagement['topic'] = user_engagement['topic'].apply(set)
        user_engagement['topic'] = user_engagement['topic'].apply(','.join);
        user_engagement['type'] = 'Directed';
        user_engagement['weight'] = user_engagement['like']*1 + user_engagement['comment']*2 + user_engagement['share']*2;
        user_engagement_total_weight = user_engagement['weight'].sum();
        user_engagement['weight'] = user_engagement['weight']*100/ user_engagement_total_weight;
        user_engagement.rename(columns={'topic':'linkage', 'uid':'source', 'page_id':'target'}, inplace=True);

        users_friends = sna.get_user_x_user();
        users_friends['linkage'] = users_friends['linkage'] +',friend';
        users_friends['type'] = 'Directed';
        users_friends['weight'] = 0.01;

        page_propogation = sna.get_user_x_page();
        page_propogation.drop(columns=['id'], inplace=True);
        page_propogation['linkage'] = 'ads-propogation';
        page_propogation['type'] = 'Directed';
        page_propogation['weight'] = 0.05;
        page_propogation.rename(columns={'page_id':'source', 'uid':'target'},inplace=True);

        edge = pd.concat([users_friends, user_engagement, page_propogation]);
        edge.to_csv('edge.csv', index=False);

        # Gen node
        users = sna.get_users();
        users['type']='user';
        users.rename(columns={'uid':'id'}, inplace=True);
        users.drop(columns=['topic', 'topic_p'], inplace=True);

        pages = sna.get_pages()
        brands = sna.get_brands()
        pages = pages.set_index('brand').join(brands[['id','mrkt_value']].set_index('id'));
        pages.reset_index(names=['brand'], inplace=True);

        node = pd.concat([users, pages]);
        node.to_csv('node.csv', index=False);
        pass;

    def gen_brand_engagement(self):
        sna = self.sna;

        interact = sna.get_post_interaction();
        posts = sna.get_posts();
        brands = sna.get_brands()[['id', 'type']];
        brands.rename(columns={'type':'brand_type'}, inplace=True);
        pages = sna.get_pages()[['id', 'type']];
        pages.rename(columns={'type':'page_type'}, inplace=True);

        uxu = sna.get_user_x_user();
        uxp = sna.get_user_x_page();

        # Gen edge
        interact['score'] = interact['like'] + 2*interact['comment'] + 2*interact['share'];
        interact['score'] = interact['score']/100;
        interact.drop(columns=['id', 'like', 'comment', 'share'], inplace=True);

        interact = interact.set_index('post_id').join(posts.set_index('id')).reset_index(names=['post_id']);
        interact.drop(interact[~interact.brand.isin(brands.id)].index, inplace=True);

        interact = interact.set_index('brand').join(brands.set_index('id')).reset_index(names=['brand']);
        interact = interact.set_index('page_id').join(pages.set_index('id')).reset_index(names=['page_id']);

        ## User-brand engagment
        user_brand = interact.groupby(['uid', 'brand', 'brand_type'])['score'].sum().reset_index();
        tmp = interact.groupby(['uid', 'brand', 'brand_type'])['topic'].apply(set).reset_index();
        tmp['topic'] = tmp['topic'].apply(lambda x: ','.join(x));
        user_brand = user_brand.set_index(['uid', 'brand', 'brand_type']).join(
                tmp.set_index(['uid', 'brand', 'brand_type'])).reset_index();

        user_brand['src_type'] = 'user';
        user_brand['type'] = 'undirected';
        user_brand['relation'] = 'user-engagement';
        user_brand.rename(columns={'uid':'source', 'brand':'target', 'score':'weight', 'brand_type':'tgt_type'}, inplace=True);

        ## page-brand engagement
        page_brand = interact.groupby(['page_id', 'brand', 'brand_type', 'page_type'])['score'].sum().reset_index();
        tmp = interact.groupby(['page_id', 'brand', 'brand_type', 'page_type'])['topic'].apply(set).reset_index();
        tmp['topic'] = tmp['topic'].apply(lambda x: ','.join(x));
        page_brand = page_brand.set_index(['page_id', 'brand', 'brand_type', 'page_type']).join(
                tmp.set_index(['page_id', 'brand', 'brand_type', 'page_type'])).reset_index();
        page_brand['type'] = 'undirected';
        page_brand['relation'] = 'inflencer-engagement';
        page_brand.rename(columns={'page_id':'source', 'brand':'target',
                'page_type':'src_type', 'brand_type':'tgt_type', 'score':'weight'}, inplace=True);

        ## user-friend relation
        uxu.drop(uxu[uxu.linkage=='non-mu-topic'].index, inplace=True);
        uxu['src_type'] = 'user';
        uxu['tgt_type'] = 'user';
        uxu['weight'] = 0.01;
        uxu['type'] = 'undirected';
        uxu['relation'] = 'user-user';
        uxu.rename(columns={'linkage':'topic'},inplace=True);

        ## user-page-follow relation
        uxp.drop(columns=['id'], inplace=True);
        uxp['src_type'] = 'user';
        uxp = uxp.set_index('page_id').join(pages.set_index('id')).reset_index(names=['page_id']);
        uxp['weight'] = 0.01;
        uxp['type'] = 'undirected';
        uxp['relation'] = 'user-page';

        tmp = interact.groupby(['uid','page_id'])['topic'].apply(set).reset_index();
        uxp = uxp.set_index(['uid', 'page_id']).join(tmp.set_index(['uid','page_id'])).reset_index();
        uxp.dropna(subset=['topic'],inplace=True);
        uxp['topic'] = uxp['topic'].apply(','.join);
        uxp.rename(columns={'uid':'source', 'page_id':'target', 'page_type':'tgt_type'},inplace=True);

        edge = pd.concat([user_brand, page_brand, uxu, uxp]);
        edge.to_csv('edge.csv', index=False);

        #Gen node
        node = sna.get_users()[['uid']].loc[:];
        node['type'] = 'user'
        node.rename(columns={'uid':'id'}, inplace=True);
        pages.rename(columns={'page_type':'type'},inplace=True);
        brands = sna.get_brands();
        node = pd.concat([node, pages, brands]);
        node.to_csv('node.csv', index=False);
        pass;

GenGraph().gen_brand_engagement()